In [1]:
# %pip install torch
import sys
print(sys.executable)
import torch

/home/eeyad/projects/miniconda3/envs/lab/bin/python


In [2]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 16 # what is the maximum context length for predictions?
max_iters =  5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 30 # 200
n_embd = 32
n_layer = 2
n_head = 2
dropout = 0.2

In [4]:
with open('quran-simple.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
len(text)

735382

In [6]:
og_vocab = sorted(list(set(text)))
print(''.join(og_vocab))
print(len(og_vocab))
std_vocab = '0123456789|ءآأؤإئابةتثجحخدذرزسشصضطظعغـفقكلمنهوىي ۩'
std_vocab_size = len(std_vocab)
print(std_vocab_size)

reveries_vocab = 'ًٌٍَُِّْ'


 0123456789|ءآأؤإئابةتثجحخدذرزسشصضطظعغـفقكلمنهوىيًٌٍَُِّْٰۖۗۘۙۚۛۜ۩
67
50


In [7]:
text[:100]

'1|1|بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ\n1|2|الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ\n1|3|الرَّحْمَـ'

In [8]:
def NextToken(index, text):
    if text[index] in numerics:
        token = [text[index],]
        j = index + 1
        while j < len(text) and text[index] in numerics:
            token.append(text[j])
            j += 1
            if text[j-1] == '|':
                break
        return ''.join(token), j
    elif text[index] in std_vocab:
        token = [text[index],]
        j = index + 1
        while j < len(text) and text[j] in reveries_vocab:
            token.append(text[j])
            j += 1
        return ''.join(token), j
    else:
        return text[index], index + 1

In [9]:
total = []
numerics = '0123456789|'
i = 0
test_text = text # text[:1000000]
while i < len(test_text):
    token, i = NextToken(i, test_text)
    # print(token)
    total.append(token)
print(len(total))

436583


In [ ]:
actual_vocab = sorted(list(set(total)))
actual_vocab_size = len(actual_vocab)
actual_vocab_size, actual_vocab

In [11]:
stoi = {ch:i for i,ch in enumerate(actual_vocab)}
itos = {i:ch for i,ch in enumerate(actual_vocab)}
# encode = lambda s: [stoi[ss] for ss in s]
def encode(text):
    i = 0
    total = []
    while i < len(text): 
        token, i = NextToken(i, text)
        total.append(stoi[token])
    return total
decode = lambda i: ''.join([itos[ii] for ii in i])
"إِيَادْ إِبْرَاهِيمْ", decode(encode("إِيَادْ إِبْرَاهِيمْ"))



('إِيَادْ إِبْرَاهِيمْ', 'إِيَادْ إِبْرَاهِيمْ')

In [142]:
# text.translate({ord(c): (c if c in vocab else None) for c in vocab})[:10]
# Not working for some reason.

TypeError: ord() expected a character, but string of length 4 found

In [12]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:1000])

tensor([112, 112, 321, 450, 586,   1, 314, 566, 576, 613,   1, 314, 566, 424,
        379, 584, 524, 645, 600,   1, 314, 566, 424, 376, 632, 586,   0, 112,
        210, 314, 579, 374, 593, 394,   1, 572, 576, 613,   1, 418, 327,   1,
        314, 579, 509, 314, 570, 586, 632, 598,   0, 112, 221, 314, 566, 424,
        379, 584, 524, 645, 600,   1, 314, 566, 424, 376, 632, 586,   0, 112,
        232, 584, 314, 572, 558,   1, 635, 630, 586,   1, 314, 566, 401, 632,
        600,   0, 112, 243, 307, 641, 314, 556,   1, 598, 516, 320, 394,   1,
        621, 307, 641, 314, 556,   1, 598, 450, 339, 511, 632, 599,   0, 112,
        254, 314, 617, 395, 598, 314,   1, 314, 566, 470, 418, 314, 486,   1,
        314, 579, 585, 450, 339, 544, 632, 584,   0, 112, 265, 467, 418, 314,
        486,   1, 314, 576, 409, 632, 598,   1, 297, 606, 509, 593, 339,   1,
        509, 570, 644, 613, 593,   1, 520, 644, 420,   1, 314, 579, 584, 523,
        476, 618, 321,   1, 509, 570, 644, 613, 593,   1, 621, 5

In [13]:
n = int(0.9 * len(data))
# revisit this. Quran is complicated and end is probably the shorter verses which means training and validation are already diverting.
train_data = data[:n]
val_data = data[n:]

In [14]:
block_size = 8
train_data[:block_size + 1]

tensor([112, 112, 321, 450, 586,   1, 314, 566, 576])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
x, y
context = []
for t in range(block_size):
    context = x[:t+1]
    print('input: ', context, ' and target: ', y[t])

input:  tensor([112])  and target:  tensor(112)
input:  tensor([112, 112])  and target:  tensor(321)
input:  tensor([112, 112, 321])  and target:  tensor(450)
input:  tensor([112, 112, 321, 450])  and target:  tensor(586)
input:  tensor([112, 112, 321, 450, 586])  and target:  tensor(1)
input:  tensor([112, 112, 321, 450, 586,   1])  and target:  tensor(314)
input:  tensor([112, 112, 321, 450, 586,   1, 314])  and target:  tensor(566)
input:  tensor([112, 112, 321, 450, 586,   1, 314, 566])  and target:  tensor(576)


In [17]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[  1, 646,   1, 621, 314, 342, 543, 618],
        [618, 598,   1, 529, 341, 632, 567, 314],
        [585, 618, 443, 631, 645,   0, 123, 241],
        [  1, 529, 570, 314,   1, 339, 413, 611]])
targets:
torch.Size([4, 8])
tensor([[646,   1, 621, 314, 342, 543, 618, 314],
        [598,   1, 529, 341, 632, 567, 314,   0],
        [618, 443, 631, 645,   0, 123, 241, 529],
        [529, 570, 314,   1, 339, 413, 611, 328]])
----
when input is [1] the target: 646
when input is [1, 646] the target: 1
when input is [1, 646, 1] the target: 621
when input is [1, 646, 1, 621] the target: 314
when input is [1, 646, 1, 621, 314] the target: 342
when input is [1, 646, 1, 621, 314, 342] the target: 543
when input is [1, 646, 1, 621, 314, 342, 543] the target: 618
when input is [1, 646, 1, 621, 314, 342, 543, 618] the target: 314
when input is [618] the target: 598
when input is [618, 598] the target: 1
when input is [618, 598, 1] the target: 529
when input is [618, 5

In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(4337)

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out
    
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    
class FeedForward(nn.Module):
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        self.sa = MultiHeadAttention(n_head, n_embd // n_head)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
        

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(actual_vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.f_ln = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, actual_vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.f_ln(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



    

In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X = X.to(device)
            Y = Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [24]:
model = BigramLanguageModel()
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
print('running on %s' % device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 6.6282, val loss 6.6065
step 500: train loss 4.2329, val loss 4.2826
step 1000: train loss 3.9348, val loss 4.0533
step 1500: train loss 3.7482, val loss 3.8845
step 2000: train loss 3.5680, val loss 3.7648
step 2500: train loss 3.3012, val loss 3.5405
step 3000: train loss 3.3508, val loss 3.5407
step 3500: train loss 3.2883, val loss 3.6330
step 4000: train loss 3.1544, val loss 3.4824
step 4500: train loss 3.2039, val loss 3.4884
running on cuda

18|اذٌمُابٍزٌ اللَّذِزِ108|وَأَطْو220|هُمْ مُّأْٰيَضَّي مَّاصِّي أَنْا فِي ظَ لَهُكَ عَيَثٍ أُغْبْرُمَا كَنَ فَّتَسْكَا تَمَا وْمِ الْيْنْ يَمْ مِنَّ ۖ فَهَا إِنَّ عَلَ يَدِدْنَعِ بِمَا بِلَمَا يَالْمُدَّا مُّالْأَمْ تَمُتَ هُوَيُقِا أَرْهَّهُ فَاوَادُ مِنْا أَبِهَا حَذْنَّالِدِمْ ۖ وَتَرَبُّ إِفَاكَا إِنْا مَهُمْ كُمْ بِدِحِمُ 66|لْعَأُودَّا حْآيَوْهِ لَ رُورَوْ أَ56|يَقَافَؤُسَأَن اللَّادِصَوْهَ نُبَجْلَا خَونَ فِيوَالرْظُنَهُم مُّينَكُذٍينَ مَحَوْيَٰ ۖ إِةِ أَعُ وَوَالَّهُمْ لَّا ۖ وَبِوَحًعُ وَآفِرَّرًا وَتَقُىٰ كَمَا

In [194]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


42|51|4|88|اللَّهُ فِي هَنِ افِيمٌ كَذَالَّطَا جَرَجَعَظِي ۖ عَذَالْكَالْمُوالنَّبْلِنَ الْبَىٰ تَيْنِ سُوتَحَكَا خَلْيَةٍ كَا مَلُونَ كَا عَلَاللَّ يُجْرِ أَوْفَرَّبِّكَهْدِ يَجِئْتَاللَّابُلًانِينَ
47|وَإِلَّهُمْرِي وَإِنَّجَّ187|فَكُرْقُصُّمَّ الْآوَقَالْتَ ا عَرْضِي وَأَعْدِ فَمَـٰ أَنَّاطِيلٍ مَّ فِلَاتَ مَ ۚ تُغْ ۖ مِنْ وَأَن ا جُرُونَ ا وَمَنُسَلْأَأَلَالَّذِيدِي إِبْخَذَٰ عِن عَلِلَّهُمْعَلَـٰلِلَّا أَعْبُوطًالْحَمِنكُمْ نَ عَ مَغْرَحْمَالْحَقَّقُرْهَ ۖ تُمْ نَ وَهُمْ ۚ تَا أَرْطَ جَوَالرۛ وَكَاللَّهُ فِيثَاللَّذِرَاءَتْرَبِّكُ بِهِ وَمَكْرٍ وَذَكَّنَّالْحِي قُوَّلِلسَّمَرَجَزَلَيْنَ
12|36|21|23|37|79|كَمَسْتَعَهُمُ
31|قُلُولُوا ادْخِيهَ وَ الرِّمَادَ أَلْمُونَصِرْضِ أَكُلُوسَطُرَّبُّكَمَا ۚ وَّلِ يَظْلِكَةَ لِقَوْلًا خَذْنَالنَّهُمْرِيلًالصَّا ۚ ذُ ضِيَهُ قَدْعُ عَلْيُن مَّعَلَّ إِنَّمُمَـٰهِ مِنتَذَٰ اللَّذِيمٌ
5|63|ا أَخَلَوْلَـٰ ا عَالْ عَلَكْرًا ۚ وَمَلُوابِيلًالَّهِ ا كَ ۚ حَا أَفَسَالنَّ تَفْتِ وَسَفَأْوَجَلِكَ قُ تَخَلَىٰكِرٍ رُكُمْ فَرِ
40|إِلَّ فَ بِبَارَ كَ امَ وَ ق

In [206]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


2|203|قَ مَلْحَمُعِلَّقَدْتِ وَإِلُوا يُجْتَشَيْنَ
16|رُّدِنَ
26|كْسَ40|الَـٰفِ خَلِيمٌ حَيَا آمَنَا اللَّهَهُ فِ نِمَّلِرَ سَلْمَدُّنُ يَغُدَّقُ طَفَزَقَاسَلُونَ آلُ نَظْ عَ90|هِمْفِ يُضٍملُونَ
27|وَكَهَا بَمَاهْ البَرُ كَامٌالْآخُخْبِخُنِ بِهِ فِهَحْهُ ۚ أَيَوْمِ يَرُودٍوا سَلَّكُمْ تَسْلٍ نَحْيَمْ مِّنًاهُ أَبْلَا مَا مُّتَيْتَنَّ بِكَالَهُ مِن الظَّ
2|63|وَأَنسِوَتِنَا الَّذِينَ
75|41|6|وَمَا ۚرُ
26|يُطَكَثِرًا مِّلُوا لِيُرًا كَا وَمَن عَنَا مَن يُسَقْيكَ
9|وَجُهِمْدُ يَـقَابٌ خُنْ يَعْتُمْ إِذَارِ فِرِّ آيَثْلِرِّيًّىٰ حَسْتَرَهُ لَىٰ فِيرْسَنِّينَ أَنتُضُونَ
دِ15|26|نَّهُم وَمُوعِينَ السَّهِي تَشَشْكَ أَلْأَرْرِيَا نَالَّهُم الْ أَقْرَىٰذَهُرْمْ آتَأْتِدَّهُ الْبَسْرَ 106|يَجْإِنَّكُ هُمْ الَّذِيمُ هُمْ وَعَفْزَالُونَ
2|18|لَّهَ تَ مُوا اتَّنُوا اللَّذِيرُ أَرْكَانَ
39|21|طَّمَا بَنَّدْمِيرٍ
15|47|وْخَ بِصِيفُ مِن يُشْينَ
34|ابٍ
43|سَتَفْ مَلُونَهَا وَقُرَ وَأَقْاوَدُونَ
9|9|47|وَقَلُونَ كَ ابَرْلْصُولْكُ124|دَبَ أَرْنِينِ وَعْ أُوا خَتَزْلَكَثِينَ
كَفُبَا ۗ وَأَناكِنَ
19|13|نٍ